## Smali & Manifest File Analysis

This notebook will try to analyze a directory and its SMALI and MANIFEST files. The analysis will find and print sensitive methods and packages used by the program. 

In [1]:
from os import walk
import pandas as pd

In [2]:
methods_to_find = []
perms_to_find = []
found_perms = []
found_lines = []

#read file using pandas
df = pd.read_csv('functions.csv', engine='python')
methods_to_find = df.functions
df = pd.read_csv('perms.csv', engine='python')
perms_to_find   = df.perms

In [3]:
def findFunctionsInDir(dir, list=0, verbose = True):
    #for each directory, and files
    for dirpath, dirnames, files in walk(dir):

        #skip the res folder, too many subfolders, slows.
        if('res' in dirpath): continue

        if(verbose == True): print(f'Found directory: {dirpath}')
        for file_name in files:

            #if it is a smali file
            if('.smali' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                with open(dirpath + "\\" + file_name) as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in methods_to_find:
                            if(i in line and 'invoke' in line):
                                #print the harmful line
                                if(line.strip() not in found_lines):
                                    if(verbose == True): print(num, ": ", line)
                                    found_lines.append(line.strip())

            #if it is a manifest file
            if('.xml' in file_name and 'anifest' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                #I had to use latin encoding here or it would simply not work.
                with open(dirpath + "\\" + file_name, encoding="latin-1") as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in perms_to_find:
                            if(i in line and ('<uses-permission android:name=' in line)
                                         or  ('<action android:name="android.app.action') in line):
                                #print the harmful line
                                if(line.strip() not in found_perms):
                                    if(verbose == True): print(num, ": ", line)
                                    found_perms.append(line.strip())

In [4]:
#directory = "C:\\Users\\vinic\\Desktop\\apktool\\simp"\
directory = "C:\\Users\\vinic\\Desktop\\output"

findFunctionsInDir(dir=directory, verbose=False)

KeyboardInterrupt: 

In [5]:
#pretty print the found lines
print('\n'.join(found_lines))

invoke-virtual {v2, v0, v1}, Landroid/content/Context;->getSharedPreferences(Ljava/lang/String;I)Landroid/content/SharedPreferences;
invoke-virtual {v2, v1, v0}, Landroid/content/Context;->getSharedPreferences(Ljava/lang/String;I)Landroid/content/SharedPreferences;
invoke-virtual {p0}, Landroid/content/Context;->getFilesDir()Ljava/io/File;
invoke-direct {v3, v1}, Ljava/io/FileInputStream;-><init>(Ljava/io/File;)V
invoke-virtual {v1, v0, v2}, Landroid/content/Context;->getSharedPreferences(Ljava/lang/String;I)Landroid/content/SharedPreferences;
invoke-direct {v6, v5}, Ljava/io/FileInputStream;-><init>(Ljava/io/File;)V
invoke-direct {v2, v4}, Ljava/io/FileInputStream;-><init>(Ljava/io/File;)V
invoke-virtual {v0}, Landroid/content/Context;->getFilesDir()Ljava/io/File;
invoke-virtual {v1, v0, v4}, Landroid/content/Context;->getSharedPreferences(Ljava/lang/String;I)Landroid/content/SharedPreferences;
invoke-direct {v0, v11}, Ljava/io/FileInputStream;-><init>(Ljava/io/File;)V
invoke-direct {

In [ ]:
#pretty print found permissions and actions
print('\n'.join(found_perms))

<uses-permission android:name="android.permission.READ_PHONE_STATE"/>
<uses-permission android:name="android.permission.RECEIVE_BOOT_COMPLETED"/>
<uses-permission android:name="android.permission.WRITE_EXTERNAL_STORAGE"/>
<uses-permission android:name="android.permission.READ_CONTACTS"/>
<uses-permission android:name="android.permission.CALL_PHONE"/>
<uses-permission android:name="android.permission.SYSTEM_ALERT_WINDOW"/>
<uses-permission android:name="android.permission.GET_TASKS"/>
<uses-permission android:name="android.permission.READ_SMS"/>
<uses-permission android:name="android.permission.SEND_SMS"/>
<uses-permission android:name="android.permission.RECEIVE_SMS"/>
<action android:name="android.app.action.ACTION_DEVICE_ADMIN_DISABLE_REQUESTED"/>
<action android:name="android.app.action.DEVICE_ADMIN_ENABLED"/>
<action android:name="android.app.action.DEVICE_ADMIN_DISABLED"/>


In [ ]:
#TODO: split 2 search functionalities
#TODO: Add a function counter
#TODO: add a GUI
#TODO: add a way to select path
#TODO: Increase the search list
#TODO: Print results in a better way
#TODO: output to a file.